<a href="https://colab.research.google.com/github/Riipou/First-order/blob/main/Projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [46]:
import numpy as np
from scipy.io import loadmat
import numpy as np
import csv

# Chargement et pré-traitement des données

In [47]:
def one_hot_encoding(y):
    new_y = []
    for label in y:
        label_vector = np.zeros(20)
        label_vector[label - 1] = 1
        new_y.append(label_vector)
    y = np.array(new_y)
    return y


def shuffle(X, y):
    index = np.arange(np.shape(X)[1])
    np.random.shuffle(index)
    X = X[:, index]
    y = y.T[:, index]
    return X, y


def split_data(X, y, split_value=0.8):
    m = X.shape[1]
    m = int(m * split_value)
    X_train = X[:, :m]
    y_train = y[:m, :]
    X_test = X[:, m:]
    y_test = y[m:, :]
    return X_train, y_train, X_test, y_test


In [48]:
# Charger le fichier .mat
nom_fichier_mat = 'data_doc.mat'
donnees = loadmat(nom_fichier_mat)

# Xts combien de fois le mot dans le texte
if 'Xts' in donnees:
    Xts = donnees['Xts']
else:
    print("La variable 'Xts' n'a pas été trouvée dans le fichier.")

# id texte avec classe
if 'yts' in donnees:
    yts = donnees['yts']
else:
    print("La variable 'yts' n'a pas été trouvée dans le fichier.")

# Xvr combien de fois le mot dans le texte
if 'Xvr' in donnees:
    Xvr = donnees['Xvr']
else:
    print("La variable 'Xvr' n'a pas été trouvée dans le fichier.")

yts = one_hot_encoding(yts)
Xts, yts = shuffle(Xts, yts)
# X_train, y_train, X_test, y_test = split_data(Xts, yts)

#Descente du gradient

In [49]:
def loss_function(y, y_pred):
    n = y.shape[1]
    loss = (np.linalg.norm(y - y_pred, 'fro') ** 2) / n
    return loss

def sigmoid(X, a):
    return 1 / (1 + np.exp(-a * X))

def sigmoid_derivate(X, a):
    return a * sigmoid(X, a) * (1 - sigmoid(X, a))

def relu(x, a):
    return np.maximum(0, a * x)

def initialization_random(m, p):
    W = np.random.rand(m, p) - 0.5
    b = np.random.rand(p) - 0.5
    return W, b

def initialization_xavier(m, p):
    W = np.random.randn(m, p) * np.sqrt(2 / (m + p))
    b = np.zeros((p,))
    return W, b

def initialization_lecun(m, p):
    W = np.random.randn(m, p) * np.sqrt(1 / m)
    b = np.zeros((p,))
    return W, b

def grad_W(W, B, a, X, y, n):
    return X @ ((1 / n) * np.multiply(sigmoid(W.T @ X + B, a) - y, sigmoid_derivate(W.T @ X + B, a))).T

def grad_B(W, B, a, X, y, n):
    v1 = np.ones((n, 1))
    return ((1 / n) * np.multiply(sigmoid(W.T @ X + B, a) - y, sigmoid_derivate(W.T @ X + B, a))) @ v1

def update_W(alpha, W, B, a, X, y):
    loss = loss_function(y, sigmoid(W.T @ X + B, a))
    alpha *= 1.5
    W2 = W - alpha * grad_W(W, B, a, X, y, n)
    while loss_function(y, sigmoid(W2.T @ X + B, a)) > loss:
        alpha /= 2
        W2 = W - alpha * grad_W(W, B, a, X, y, n)
    return W2, alpha

def update_B(alpha, W, B, a, X, y):
    loss = loss_function(y, sigmoid(W.T @ X + B, a))
    alpha *= 1.5
    b = B[:, 1]
    b = b.reshape(-1, 1)
    b2 = b - alpha * grad_B(W, B, a, X, y, n)
    B2 = np.tile(b2.reshape(-1), (n, 1)).T
    while loss_function(y, sigmoid(W.T @ X + B2, a)) > loss:
        alpha /= 2
        b2 = b - alpha * grad_B(W, B, a, X, y, n)
        B2 = np.tile(b2.reshape(-1), (n, 1)).T
    return B2, alpha


def linear_model(m, n, p, max_iterations, a, precision, X, y):

    # Initialisation random pour les poids (76,987)
    # W, b = initialization_wavier(m, p)

    # Initialisation de Xavier/Glorot pour les poids (73,513)
    # W, b = initialization_xavier(m, p)

    # Initialisation de Lecun pour les poids (77,388)
    W, b = initialization_lecun(m, p)

    B = np.tile(b, (n, 1)).T
    alpha_W = 0.1 * np.linalg.norm(W, 'fro') / np.linalg.norm(grad_W(W, B, a, X, y, n), 'fro')
    alpha_B = 0.1 * np.linalg.norm(B, 'fro') / np.linalg.norm(grad_B(W, B, a, X, y, n), 'fro')
    for _ in range(max_iterations):
        loss_prev = loss_function(y, sigmoid(W.T @ X + B, 1))
        W, alpha_W = update_W(alpha_W, W, B, a, X, y)
        B, alpha_B = update_B(alpha_B, W, B, a, X, y)
        loss = loss_function(y, sigmoid(W.T @ X + B, 1))
        print(loss)
        if abs(loss - loss_prev) < precision:
            print("break")
            break
    return W, b



# Descente de gradient

In [50]:
m, n = Xts.shape
p, _ = yts.shape

num_iterations=100000
a = 0.05
precision = 1e-5


W, b = linear_model(m, n, p, num_iterations, a, precision, Xts, yts)

3.831677972104292
2.8363168033789137
2.110804191686314
1.6493022510372604
1.3803475597566905
1.225792918882283
1.1335174922456834
1.0771798361000262
1.042783524750171
1.0214312268326529


<ipython-input-49-37600a3db8f4>:7: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-a * X))


1.0069197431640378
0.995678071416539
0.984350405302947
0.970804096059198
0.9503465667040029
0.9243904744198863
0.8889434396514099
0.842502541917613
0.7703020243523353
0.6737501691984503
0.5842818712830755
0.5215079159960435
0.48583904119267024
0.4472965451169802
0.45869477899502353
0.43533420022842995
0.4002165264767054
0.39619950942114407
0.33138824621340984
0.29821984680058355
0.24599778055056162
0.25945943238488106
0.23200010832342136
0.24315280425555072
0.20020479465901128
0.2023653377874734
0.18969269375187056
0.17461470898090936
0.164239522678413
0.16553908987925922
0.15500583801288983
0.15969765656530432
0.14984858272166793
0.14282993311080613
0.14207544820899864
0.13772449652346266
0.14112840838788396
0.1372606701224287
0.13354489313880993
0.13124495960720292
0.12988750549514466
0.12684125257709578
0.12666733534872288
0.1242174479721127
0.12446243752962766
0.12231267135189103
0.12394913214631943
0.12001446004065335
0.12094306746518
0.11822686625780553
0.11971198570004955
0.1170

# Evaluation des données

# Création validation

In [51]:
def reverse_one_hot_encoding(y):
    y = np.argmax(y, axis=0)+1
    return y

def test(X, W, b):
    _, n = X.shape
    B = np.tile(b, (n, 1)).T
    y = sigmoid(W.T @ X + B, 1)
    y = reverse_one_hot_encoding(y)
    ids = np.arange(1, len(y)+1)
    data = np.column_stack((ids, 1+100 * y))
    nom_fichier = "test2.csv"

    header = ["id","class"]
    with open(nom_fichier, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header)
        writer.writerows(data)
    return 0

In [52]:
test(Xvr, W, b)

<ipython-input-49-37600a3db8f4>:7: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-a * X))


0